In [1]:
import tweepy
from nltk.tokenize import WhitespaceTokenizer

import csv
import pandas as pd
import json
import requests
from TwitterAPI import TwitterAPI
import time
import numpy as np
import matplotlib.pyplot as plt

# Create a tokenizer from NLTK which will create tokens based on the whitespace in between words
tokenizer = WhitespaceTokenizer()
#Twitter API credentials
consumer_key = "34EB0NsOyALkliVWWQhcomlcq"
consumer_secret = "SdjamaQ3Nf479eMi1ke13tgnYuOAJrkms3zgdBwgctmWVIdeSC"
access_key = "848683524236484609-0S5xPYghok2MOav3tUKbhQkVcJtAH0u"
access_secret = "FhnJ2rTyVMFlepLd1Osnz9DZhJnNuq0td4XxCczojQiy9"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
twitter_api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

twtapi = TwitterAPI(consumer_key, consumer_secret, access_key, access_secret)

last_id_1 = -1

In [ ]:
#Extract tweets for the link provided  
def get_all_tweets(fakenews,sn):

    searched_tweets=[]
    last_id = -1
    count=100

    #First extraction before looping
    new_tweets = api.search(q=fakenews, count=count, max_id=str(last_id - 1))
    searched_tweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    last_id = new_tweets[-1].id

    print ("#New Tweets:",len(new_tweets))

    if len(new_tweets)<99:
        max_tweets=len(new_tweets)
    else:
        max_tweets=100

    while len(searched_tweets) < max_tweets:
        count = max_tweets - len(searched_tweets)
        print ("#Extracted tweets:",len(searched_tweets))
        try:
            new_tweets = api.search(q=fakenews, count=count, max_id=str(last_id - 1))
            if not new_tweets:
                break
            searched_tweets.extend(new_tweets)
            last_id = new_tweets[-1].id

        except tweepy.TweepError as e:
            print ("Error")
            break

    #transform the tweepy tweets into a 2D array that w ill populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"),tweet.author._json['screen_name'],tweet.author._json['followers_count']] for tweet in searched_tweets]

    #transform the tweepy tweets into a 2D array that will populate the csv
    with open('%s_tweets.csv' % "trial", 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text","retweet handle","followers count"])
        writer.writerows(outtweets)
    pass

    tweets_df=pd.read_csv("trial_tweets.csv")

    bot_score_lst=[]
    for screen_name in tweets_df["retweet handle"]:
        user_timeline = twitter_api.user_timeline(screen_name, count=200)

        if user_timeline:
            user_data = user_timeline[0]['user']
        else:
            user_data = twitter_api.get_user(screen_name)

        if not screen_name.startswith('@'):
            screen_name = '@' + screen_name
        search = twitter_api.search(screen_name, count=100)
        mentions = search['statuses']

        tweets = user_timeline + mentions
        len(tweets)

        post_body = {'content': tweets,
                     'meta': {
                        'user_id': user_data['id_str'],
                        'screen_name': screen_name,
                        },
                    }

        bon_url = 'http://truthy.indiana.edu/botornot/api/1/check_account'

        bon_response = requests.post(bon_url, data=json.dumps(post_body))

        #bon_response.status_code

        #bon_response.json()
        bot_score_lst.append(bon_response.json()["score"])

    tweets_df["bot_score"]=bot_score_lst
    bot_count=tweets_df[(tweets_df["bot_score"]>0.5)].count()["bot_score"]
    notabot_count=tweets_df[(tweets_df["bot_score"]<0.5)].count()["bot_score"]
    bot_followers=tweets_df[(tweets_df["bot_score"]>0.5)]["followers count"].sum()
    notabot_followers=tweets_df[(tweets_df["bot_score"]<0.5)]["followers count"].sum()

    print "Bot count:",bot_count
    print "Not a Bot count:",notabot_count
    print "Bot followers:",bot_followers
    print "Not a Bot followers:",notabot_followers


    #Tweet the following stats on twitter
    twt_txt1= "I analyzed the news and observed that"

    if bot_count>notabot_count:
        twt_txt2="bots shared it more than real individuals"
    else:
        twt_txt2="real individuals shared it more than bots"

    twt_txt3= "\nStatistics are shared below:"

    status="Hey @%s, %s, %s.%s" %(sn,twt_txt1,twt_txt2,twt_txt3)

    objects = ('Bot', 'Not a Bot')
    y_pos = np.arange(len(objects))
    count = [bot_count,notabot_count]

    plt.figure(figsize=(5,5))    
    plt.bar(y_pos, count, align='center', alpha=0.9, width=0.4)
    plt.xticks(y_pos, objects)
    plt.ylabel('Count')
    plt.title('Shares on twitter')
    #plt.show()
    plt.savefig('Vizdisp/bon.png', bbox_inches='tight')

    objects = ('Bot', 'Not a Bot')
    y_pos = np.arange(len(objects))
    count = [bot_followers,notabot_followers]

    plt.figure(figsize=(5,5))    
    plt.bar(y_pos, count, align='center', alpha=0.9, width=0.4)
    plt.xticks(y_pos, objects)
    plt.ylabel('Count')
    plt.title('Potential Reach based on #Followers')
    #plt.show()
    plt.savefig('Vizdisp/bon_fol.png', bbox_inches='tight')

    x = []

    file = open('Vizdisp/bon.png', 'rb')
    data = file.read()
    r = twtapi.request('media/upload', None, {'media': data})
    media_id = r.json()['media_id']
    print('Appended Image 1 Successfully' if r.status_code == 200 else 'UPLOAD MEDIA FAILURE')
    x.append(str(media_id))


    file = open('Vizdisp/bon_fol.png', 'rb')
    data1 = file.read()
    r = twtapi.request('media/upload', None, {'media': data1})
    media_id = r.json()['media_id']
    print('Appended Image 2 Successfully' if r.status_code == 200 else 'UPLOAD MEDIA FAILURE')
    x.append(str(media_id))

    if r.status_code == 200:
        media_id = ','.join(x)
        r = twtapi.request('statuses/update', {'status':status, 'media_ids':media_id})
        print('Updload Successful' if r.status_code == 200 else 'UPDATE STATUS FAILURE')

    tweets_df.to_csv("tweets_bon.csv")

while True:
    
    twts = api.search(q="@fakeNewsViral",max_id=str(last_id_1-1))
    if not twts:
        print "No new Tweets to be analyzed."
        time.sleep(10)
    else:
        last_id_1 = twts[-1].id

        for s in twts:
            print s.text.split('|')[1]
            extracted_txt=s.text.split('|')[1]
            sn = s.user.screen_name
            get_all_tweets(extracted_txt,sn)

 World Gets Behind Putin’s Vow To Destroy New World Order
('#New Tweets:', 79)
Bot count: 12
Not a Bot count: 67
Bot followers: 61645
Not a Bot followers: 119190
Appended Image 1 Successfully
Appended Image 2 Successfully
Updload Successful
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tweets to be analyzed.
No new Tw